## Training using Neural Networks

In [1]:
import numpy as np
import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_pickle(r'train_feat.pkl')
test_df = pd.read_pickle(r'test_feat.pkl')

In [3]:
test_raw = pd.read_pickle(r'test_df.pkl')
test_raw["totals.transactionRevenue"] = test_raw["totals.transactionRevenue"].astype('float')
test_raw['totals.transactions'].fillna(0, inplace=True)
true_rev = test_raw.groupby('fullVisitorId')[["totals.transactionRevenue"]].sum().apply(np.log1p, axis=1).reset_index()

In [4]:
target = ['fullVisitorId', 'revisit', 'trans_revenue']

In [22]:
train_clf = train_df.drop(target, axis=1)
test = test_df.drop(target, axis=1)

### Using MLP Architecture

#### Building & Training MLP for Classification

In [6]:
batch_size = 64
epochs = 100
learning_rate = 0.0003

In [11]:
clf = Sequential()
clf.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', input_dim=35))
clf.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
clf.add(Dense(1, activation='sigmoid'))

In [12]:
opt = optimizers.Adam(lr = learning_rate)
clf.compile(loss='binary_crossentropy', optimizer=opt)

In [13]:
clf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               9216      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 42,241
Trainable params: 42,241
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = clf.fit(x = np.asarray(train_clf).astype(np.float32), y = train_df['revisit'],
                    batch_size = batch_size, epochs=15, verbose=1)

Epoch 1/15
26693/26693 [==============================] - 52s 2ms/step - loss: 1.8541
Epoch 2/15
26693/26693 [==============================] - 47s 2ms/step - loss: 0.0718
Epoch 3/15
26693/26693 [==============================] - 47s 2ms/step - loss: 0.0512
Epoch 4/15
26693/26693 [==============================] - 49s 2ms/step - loss: 0.0499
Epoch 5/15
26693/26693 [==============================] - 48s 2ms/step - loss: 0.0491
Epoch 6/15
26693/26693 [==============================] - 50s 2ms/step - loss: 0.0485
Epoch 7/15
26693/26693 [==============================] - 41s 2ms/step - loss: 0.0480
Epoch 8/15
26693/26693 [==============================] - 35s 1ms/step - loss: 0.0476
Epoch 9/15
26693/26693 [==============================] - 40s 2ms/step - loss: 0.0477
Epoch 10/15
26693/26693 [==============================] - 48s 2ms/step - loss: 0.0474
Epoch 11/15
26693/26693 [==============================] - 46s 2ms/step - loss: 0.0471
Epoch 12/15
26693/26693 [===========================

#### Building & Training MLP for Regression

In [23]:
train_reg = train_df.drop(target, axis=1)[train_df['revisit']==1]

In [17]:
reg = Sequential()
reg.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', input_dim=35))
reg.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
reg.add(Dense(1))

In [18]:
opt = optimizers.Adam(lr=learning_rate)
reg.compile(loss='mse', optimizer=opt)

In [19]:
reg.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               9216      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 42,241
Trainable params: 42,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = reg.fit(x = np.asarray(train_reg).astype(np.float32), y = train_df['trans_revenue'][train_df['revisit']==1],
                    batch_size = batch_size, epochs = epochs, verbose = 1)

Epoch 1/100
433/433 [==============================] - 1s 2ms/step - loss: 138470.1719
Epoch 2/100
433/433 [==============================] - 1s 2ms/step - loss: 88112.7578
Epoch 3/100
433/433 [==============================] - 1s 2ms/step - loss: 19778.9434
Epoch 4/100
433/433 [==============================] - 1s 2ms/step - loss: 3949.0525
Epoch 5/100
433/433 [==============================] - 1s 2ms/step - loss: 10952.3193
Epoch 6/100
433/433 [==============================] - 1s 2ms/step - loss: 23476.0156
Epoch 7/100
433/433 [==============================] - 1s 2ms/step - loss: 3543.0352
Epoch 8/100
433/433 [==============================] - 1s 2ms/step - loss: 14564.0693
Epoch 9/100
433/433 [==============================] - 1s 2ms/step - loss: 21842.8086
Epoch 10/100
433/433 [==============================] - 1s 2ms/step - loss: 21039.9805
Epoch 11/100
433/433 [==============================] - 1s 2ms/step - loss: 7349.8794
Epoch 12/100
433/433 [==============================] 

#### Calculating Final Predictions

In [24]:
clf_pred = clf.predict(np.asarray(test).astype(np.float32))
reg_pred = reg.predict(np.asarray(test).astype(np.float32))
reg_pred[reg_pred < 0] = 0

final_pred = clf_pred * reg_pred

In [25]:
pred_df = pd.DataFrame({"fullVisitorId":test_df["fullVisitorId"].values})
pred_df["PredictedLogRevenue"] = final_pred
pred_df.columns = ["fullVisitorId", "PredictedLogRevenue"]

#### RMSE

In [27]:
res = pd.merge(pred_df, true_rev , left_on='fullVisitorId', right_on='fullVisitorId') 
rms = np.sqrt(mean_squared_error(res['totals.transactionRevenue'], res['PredictedLogRevenue']))
print("RMSE: %.3f" %rms)

RMSE: 2.288


### Using CNN Architecture

#### Building & Training CNN for Classification

In [6]:
n_feat = 1
n_steps = 35
batch_size = 64

In [7]:
train_clf = train_df.drop(target, axis=1)
train_clf = train_clf.values.reshape((train_clf.shape[0], train_clf.shape[1], n_feat))

test = test_df.drop(target, axis=1)
test = test.values.reshape((test.shape[0], test.shape[1], n_feat))

In [8]:
clf = keras.Sequential([
                        keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_feat)),
                        keras.layers.MaxPooling1D(pool_size=2),
                        keras.layers.Flatten(),
                        keras.layers.Dense(50, activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')
                        ])

clf.compile(optimizer='adam', loss='binary_crossentropy')

In [9]:
clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 34, 64)            192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 17, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1088)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                54450     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 54,693
Trainable params: 54,693
Non-trainable params: 0
_________________________________________________________________


In [10]:
clf.fit(x = np.asarray(train_clf).astype(np.float32), y = train_df['revisit'],
                    batch_size=64, epochs=20)

Epoch 1/20
26693/26693 [==============================] - 103s 4ms/step - loss: 0.4502
Epoch 2/20
26693/26693 [==============================] - 95s 4ms/step - loss: 0.0467
Epoch 3/20
26693/26693 [==============================] - 95s 4ms/step - loss: 0.0451
Epoch 4/20
26693/26693 [==============================] - 96s 4ms/step - loss: 0.0446
Epoch 5/20
26693/26693 [==============================] - 80s 3ms/step - loss: 0.0446
Epoch 6/20
26693/26693 [==============================] - 75s 3ms/step - loss: 0.0444
Epoch 7/20
26693/26693 [==============================] - 73s 3ms/step - loss: 0.0442
Epoch 8/20
26693/26693 [==============================] - 73s 3ms/step - loss: 0.0442
Epoch 9/20
26693/26693 [==============================] - 74s 3ms/step - loss: 0.0441
Epoch 10/20
26693/26693 [==============================] - 73s 3ms/step - loss: 0.0440
Epoch 11/20
26693/26693 [==============================] - 73s 3ms/step - loss: 0.0439
Epoch 12/20
26693/26693 [==========================

#### Building & Training CNN for Regression

In [11]:
train_reg = train_df.drop(target, axis=1)[train_df['revisit']==1]
train_reg = train_reg.values.reshape((train_reg.shape[0], train_reg.shape[1], n_feat))

In [12]:
reg = keras.Sequential([
                        keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_feat)),
                        keras.layers.MaxPooling1D(pool_size=2),
                        keras.layers.Flatten(),
                        keras.layers.Dense(50, activation='relu'),
                        keras.layers.Dense(1, activation='sigmoid')
                        ])

In [13]:
reg.compile(optimizer='adam', loss='mse')

In [14]:
reg.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 34, 64)            192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 17, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1088)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                54450     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 54,693
Trainable params: 54,693
Non-trainable params: 0
_________________________________________________________________


#### Calculating Final Predictions

In [15]:
reg.fit(x = np.asarray(train_reg).astype(np.float32), y = train_df['trans_revenue'][train_df['revisit']==1],
                    batch_size=64, epochs=20)

Epoch 1/20
433/433 [==============================] - 2s 4ms/step - loss: 31.5161
Epoch 2/20
433/433 [==============================] - 2s 4ms/step - loss: 31.5161
Epoch 3/20
433/433 [==============================] - 1s 3ms/step - loss: 31.5161
Epoch 4/20
433/433 [==============================] - 1s 3ms/step - loss: 31.5161
Epoch 5/20
433/433 [==============================] - 2s 3ms/step - loss: 31.5161
Epoch 6/20
433/433 [==============================] - 2s 3ms/step - loss: 31.5161
Epoch 7/20
433/433 [==============================] - 2s 4ms/step - loss: 31.5161
Epoch 8/20
433/433 [==============================] - 2s 3ms/step - loss: 31.5161
Epoch 9/20
433/433 [==============================] - 2s 3ms/step - loss: 31.5161
Epoch 10/20
433/433 [==============================] - 2s 3ms/step - loss: 31.5161
Epoch 11/20
433/433 [==============================] - 1s 3ms/step - loss: 31.5161
Epoch 12/20
433/433 [==============================] - 2s 4ms/step - loss: 31.5161
Epoch 13/20
4

In [16]:
clf_pred = clf.predict(np.asarray(test).astype(np.float32))
reg_pred = reg.predict(np.asarray(test).astype(np.float32))
reg_pred[reg_pred < 0] = 0

final_pred = clf_pred * reg_pred

In [17]:
pred_df = pd.DataFrame({"fullVisitorId":test_df["fullVisitorId"].values})
pred_df["PredictedLogRevenue"] = final_pred
pred_df.columns = ["fullVisitorId", "PredictedLogRevenue"]

#### RMSE

In [18]:
res = pd.merge(pred_df, true_rev , left_on='fullVisitorId', right_on='fullVisitorId') 
rms = np.sqrt(mean_squared_error(res['totals.transactionRevenue'], res['PredictedLogRevenue']))
print("RMSE: %.3f" %rms)

RMSE: 2.114
